<a href="https://colab.research.google.com/github/chrismarkella/Kaggle-access-from-Google-Colab/blob/master/sql_vs_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

import numpy as np
import pandas as pd

from getpass import getpass 

In [2]:
def access_kaggle():
    """
    Access Kaggle from Google Colab.
    If the /root/.kaggle does not exist then prompt for
    the username and for the Kaggle API key.
    Creates the kaggle.json access file in the /root/.kaggle/ folder. 
    """
    KAGGLE_ROOT = os.path.join('/root', '.kaggle')
    KAGGLE_PATH = os.path.join(KAGGLE_ROOT, 'kaggle.json')

    if '.kaggle' not in os.listdir(path='/root'):
        user = getpass(prompt='Kaggle username: ')
        key  = getpass(prompt='Kaggle API key: ')
        
        !mkdir $KAGGLE_ROOT
        !touch $KAGGLE_PATH
        !chmod 666 $KAGGLE_PATH
        with open(KAGGLE_PATH, mode='w') as f:
            f.write('{"username":"%s", "key":"%s"}' %(user, key))
            f.close()
        !chmod 600 $KAGGLE_PATH
        del user
        del key
        success_msg = "Kaggle is successfully set up. Good to go."
        print(f'{success_msg}')

access_kaggle()


Kaggle username: ··········
Kaggle API key: ··········
Kaggle is successfully set up. Good to go.


In [3]:
!kaggle datasets download fernandol/countries-of-the-world --unzip
!mv countries\ of\ the\ world.csv countries_of_the_world.csv

df = pd.read_csv('countries_of_the_world.csv', sep=',')
df.head()

  0% 0.00/13.4k [00:00<?, ?B/s]
100% 13.4k/13.4k [00:00<00:00, 11.9MB/s]


,Country,Region,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Coastline (coast/area ratio),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Other (%),Climate,Birthrate,Deathrate,Agriculture,Industry,Service
0,Afghanistan,ASIA (EX. NEAR EAST),31056997,647500,"48,0","0,00","23,06","163,07",700.0,"36,0","3,2","12,13","0,22","87,65",1,"46,6","20,34","0,38","0,24","0,38"
1,Albania,EASTERN EUROPE,3581655,28748,"124,6","1,26","-4,93","21,52",4500.0,"86,5","71,2","21,09","4,42","74,49",3,"15,11","5,22","0,232","0,188","0,579"
2,Algeria,NORTHERN AFRICA,32930091,2381740,"13,8","0,04","-0,39",31,6000.0,"70,0","78,1","3,22","0,25","96,53",1,"17,14","4,61","0,101","0,6","0,298"
3,American Samoa,OCEANIA,57794,199,"290,4","58,29","-20,71","9,27",8000.0,"97,0","259,5",10,15,75,2,"22,46","3,27",NaN,NaN,NaN
4,Andorra,WESTERN EUROPE,71201,468,"152,1","0,00","6,6","4,05",19000.0,"100,0","497,2","2,22",0,"97,78",3,"8,71","6,25",NaN,NaN,NaN


In [4]:
df.shape

(227, 20)

In [5]:
df.columns

Index(['Country', 'Region', 'Population', 'Area (sq. mi.)',
       'Pop. Density (per sq. mi.)', 'Coastline (coast/area ratio)',
       'Net migration', 'Infant mortality (per 1000 births)',
       'GDP ($ per capita)', 'Literacy (%)', 'Phones (per 1000)', 'Arable (%)',
       'Crops (%)', 'Other (%)', 'Climate', 'Birthrate', 'Deathrate',
       'Agriculture', 'Industry', 'Service'],
      dtype='object')

In [7]:
df.columns = df.columns.str.strip().str.lower()
df.columns

Index(['country', 'region', 'population', 'area (sq. mi.)',
       'pop. density (per sq. mi.)', 'coastline (coast/area ratio)',
       'net migration', 'infant mortality (per 1000 births)',
       'gdp ($ per capita)', 'literacy (%)', 'phones (per 1000)', 'arable (%)',
       'crops (%)', 'other (%)', 'climate', 'birthrate', 'deathrate',
       'agriculture', 'industry', 'service'],
      dtype='object')

In [9]:
column_name_dict = {
    'area (sq. mi.)':'area',
    'pop. density (per sq. mi.)':'pop_density',
    'coastline (coast/area ratio)':'coastline',
    'infant mortality (per 1000 births)':'infant_mortality',
    'gdp ($ per capita)':'gdp_per_capita',
    'literacy (%)':'literacy',
    'phones (per 1000)':'phones',
    'arable (%)':'arable',
    'crops (%)':'crops',
    'other (%)':'other',
}

df.rename(columns=column_name_dict, inplace=True)
df.columns

Index(['country', 'region', 'population', 'area', 'pop_density', 'coastline',
       'net migration', 'infant_mortality', 'gdp_per_capita', 'literacy',
       'phones', 'arable', 'crops', 'other', 'climate', 'birthrate',
       'deathrate', 'agriculture', 'industry', 'service'],
      dtype='object')

In [11]:
df.columns = df.columns.map(lambda c: '_'.join(c.split()))
df.columns

Index(['country', 'region', 'population', 'area', 'pop_density', 'coastline',
       'net_migration', 'infant_mortality', 'gdp_per_capita', 'literacy',
       'phones', 'arable', 'crops', 'other', 'climate', 'birthrate',
       'deathrate', 'agriculture', 'industry', 'service'],
      dtype='object')

##SELECT


###SELECT 1
```SQL
SELECT population FROM world
  WHERE name = 'Germany'
```



In [12]:
filt_germany = (df.country == 'Germany')
df.loc[filt_germany, 'population']

Series([], Name: population, dtype: int64)

In [14]:
df.country.unique()[:10]

array(['Afghanistan ', 'Albania ', 'Algeria ', 'American Samoa ',
       'Andorra ', 'Angola ', 'Anguilla ', 'Antigua & Barbuda ',
       'Argentina ', 'Armenia '], dtype=object)

In [16]:
df.country.str.strip().unique()[:10]

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Anguilla', 'Antigua & Barbuda', 'Argentina', 'Armenia'],
      dtype=object)

In [17]:
df.country = df.country.str.strip()
df.country.unique()[:5]

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra'],
      dtype=object)

In [18]:
filt_germany = (df.country == 'Germany')
df.loc[filt_germany, 'population']

76    82422299
Name: population, dtype: int64

###SELECT 2


```SQL
SELECT name, population FROM world
  WHERE name IN ('Brazil', 'Russia', 'India', 'China');
```



In [19]:
selected_countries = [
    'Brazil',
    'Russia',
    'India',
    'China',
]
filt_country = (df.country.isin(selected_countries))

df.loc[filt_country, ['country', 'population']]

,country,population
27,Brazil,188078227
42,China,1313973713
94,India,1095351995
169,Russia,142893540


###SELECT 3


```SQL
SELECT name, area FROM world
  WHERE area BETWEEN 250000 AND 300000
```



In [20]:
filt_area = df.area.between(250*10**3, 300*10**3)

selected_columns = [
    'country',
    'area',
]

df.loc[filt_area, selected_columns]

,country,area
31,Burkina Faso,274200
59,Ecuador,283560
72,Gabon,267667
149,New Zealand,268680
162,Philippines,300000
223,Western Sahara,266000
